# Load libraries and Themes

In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat)
        library(ggplot2)
        library(tidyverse)
        library(presto)
        library(SeuratDisk)
        library(DESeq2)
        library(dplyr)
        library(EnhancedVolcano)
        library(Rsamtools)
        library(svglite)
        library(viridis)
        library(pals)
        })})

In [ ]:
#Color Palettes

palette.doublets <- c(
    '#197278',#turqu
    '#C1C1C1'#grey
)

palette.16 <- DiscretePalette(16, palette = "stepped", shuffle = TRUE)

palette.2 <- c(
    "#990902",
    "#0303a3"
)

palette.4 <- DiscretePalette(4, palette = "stepped", shuffle = TRUE)

palette.12 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette.15 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)

palette.dbl <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#000000",#dbl
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)

palette.ex <- c(
    "#bababa",#BC
    "#bababa",#CM_0
    "#bababa",#CM_1
    "#bababa",#CM_2
    "#bababa",#CM_3
    "#bababa",#Cyc
    "#bababa",#EC-cap
    "#000000",#ex
    "#bababa",#FB
    "#bababa",#MESO
    "#bababa",#MP_0
    "#bababa",#MP_1
    "#bababa",#NEU
    "#bababa"#PER
)
    
palette.treatment <- c(
    '#AF0000',#ALDO
    '#C1C1C1'#CTRL
)

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LA_CTRL_ALDO")

In [ ]:
set.seed(1234)

# DEG analysis ALDO vs. CTRL

In [ ]:
obj_subset <- LoadH5Seurat("seurat_objects/obj_seu_merge_harmony_sgl_addmodule_downsample_rename.h5seurat")

In [ ]:
cluster_annotations_LA <- list(
    "0" = "CM_0",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "9" = "CM_1",
    "10" = "TC",
    "11" = "EC-lym",
    "12" = "SC",
    "13" = "CM_2",
    "14" = "BC"
)

In [ ]:
obj_subset

In [ ]:
UMAP_rename_cell_type <- DimPlot(obj_subset, group.by = "cell_type")
UMAP_rename_cell_type

## Wilcoxon Rank Sum test ALDO vs. CTRL

In [ ]:
doAnalysis <- function(LA_LV, cluster_annotations){
    seurat_table = subset(x = obj_subset, subset = chamber == LA_LV)
   
    #Assign "treatment" as ident
    Idents(seurat_table) <- seurat_table@meta.data$"treatment"
        
    table_list <- lapply(names(cluster_annotations), function(cl){
        name_of_clust = cluster_annotations[[cl]]
        my_subseur <- subset(x = seurat_table, subset = seurat_clusters == cl)
        my_subseur <- NormalizeData(my_subseur)
        my_subseur <- ScaleData(my_subseur)
        my_markers <- FindMarkers(my_subseur, ident.1 = "ALDO", ident.2 = "CTRL", assay = "RNA", test.use = "wilcox", logfc.threshold = 0.1)
        write.csv(my_markers, file = paste0("DEGs/ALDOvsCTRL WRSum/degs_wilcox", "_", LA_LV, "_", name_of_clust, ".csv"))
        return(my_markers)
    })
    names(table_list) <- cluster_annotations
    
    return(table_list)
}

In [ ]:
doAnalysis("LA", cluster_annotations_LA)

### filter mitochondrial genes (not needed here)

In [ ]:
# mt_genes.csv is a list of mt- genes with >0 counts

mt_genes <- unlist(read.csv("DEGs/mt_genes.csv", header = FALSE))
file_names <- list.files("DEGs/ALDOvsCTRL WRSum", pattern = "*.csv", full.names = TRUE)

listDEG <- lapply(file_names, function(deg_list){
    read.csv(deg_list, header = TRUE, row.names = 1)
    })

listDEG <- lapply(listDEG, function(deg_list, deg_list_filtered){
    deg_list <- cbind(deg_list, Gene_names = rownames(deg_list))
    deg_list_filtered <- deg_list %>% filter(!Gene_names %in% mt_genes)
    deg_list_filtered <- deg_list_filtered[,!names(deg_list_filtered) %in% c("Gene_names")]
    }) 

listDEG_names <- gsub("\\..*", "", gsub(".*wilcox_", "", file_names))
names(listDEG) <- listDEG_names
head(listDEG[[4]], 10)

listDEG <- sapply(names(listDEG), function(name){
    lapply(listDEG, function(deg_list){
    write.csv(listDEG[[name]], file = paste0("DEGs/ALDOvsCTRL WRSum/filtered/degs_wilcox_", name, "_filtered.csv"))
    }) })

### wilcox Volcano

In [ ]:
# w/o filtering mt-genes
file_names <- list.files("DEGs/ALDOvsCTRL WRSum/forplot", pattern = "*.csv", full.names = TRUE)
listDEG <- lapply(file_names, function(deg_list){
    read.csv(deg_list, header = TRUE, row.names = 1)
})
Volcano_names <- gsub("\\..*", "", file_names)
Volcano_names <- gsub(".*degs_wilcox_LA_", "", Volcano_names)

names(listDEG) <- Volcano_names
#listDEG
length(names(listDEG))

In [ ]:
Volcano_names

In [ ]:
#options(repr.plot.width = 20, repr.plot.height = 8, repr.plot.res = 300) 

Volcano  <- lapply(names(listDEG), function(x_name){
    x = listDEG[[x_name]]
    EnhancedVolcano(x, 
    rownames(x),
    x = "avg_log2FC", 
    y = "p_val_adj", 
    FCcutoff = 0.25,
    title = x_name, gridlines.minor = FALSE, subtitleLabSize = 0, legendPosition = "none")#, xlim = c(-10, 10))
})

Volcano_Fun <- function(Volcano, no_of_rows, no_of_cols) {
    patchwork::wrap_plots(Volcano, 
    nrow = no_of_rows, ncol = no_of_cols)
}

Volcano_Patch <- Volcano_Fun(Volcano, 2, 4)

ggsave("Plots/Volcano_WRSum_withoutfilter.svg", plot = Volcano_Patch, units = "cm", dpi = 300, width = 42, height = 25, limitsize = FALSE)

In [ ]:
# w/o filtering mt-genes, only meaningful
file_names <- list.files("DEGs/ALDOvsCTRL WRSum/forplot_meaningful", pattern = "*.csv", full.names = TRUE)
listDEG <- lapply(file_names, function(deg_list){
    read.csv(deg_list, header = TRUE, row.names = 1)
})
Volcano_names <- gsub("\\..*", "", file_names)
Volcano_names <- gsub(".*degs_wilcox_LA_", "", Volcano_names)

names(listDEG) <- Volcano_names
#listDEG
length(names(listDEG))

Volcano_names

Volcano  <- lapply(names(listDEG), function(x_name){
    x = listDEG[[x_name]]
    EnhancedVolcano(x, 
    rownames(x),
    x = "avg_log2FC", 
    y = "p_val_adj", 
    FCcutoff = 0.25,
    title = x_name, gridlines.minor = FALSE, subtitleLabSize = 0, legendPosition = "none")#, xlim = c(-10, 10))
})

Volcano_Fun <- function(Volcano, no_of_rows, no_of_cols) {
    patchwork::wrap_plots(Volcano, 
    nrow = no_of_rows, ncol = no_of_cols)
}

Volcano_Patch <- Volcano_Fun(Volcano, 2, 2)

ggsave("Plots/Volcano_WRSum_withoutfilter_meaningful.svg", plot = Volcano_Patch, units = "cm", dpi = 300, width = 21, height = 25, limitsize = FALSE)

In [ ]:
# with filtering mt-genes 
file_names <- list.files("DEGs/ALDOvsCTRL WRSum/filtered/forplot", pattern = "*.csv", full.names = TRUE)
listDEG <- lapply(file_names, function(deg_list){
    read.csv(deg_list, header = TRUE, row.names = 1)
})
Volcano_names <- gsub("\\..*", "", file_names)
Volcano_names <- gsub(".*degs_wilcox_LA_", "", Volcano_names)

names(listDEG) <- Volcano_names
#listDEG
length(names(listDEG))

Volcano_names

Volcano  <- lapply(names(listDEG), function(x_name){
    x = listDEG[[x_name]]
    EnhancedVolcano(x, 
    rownames(x),
    x = "avg_log2FC", 
    y = "p_val_adj", 
    FCcutoff = 0.25,
    title = x_name, gridlines.minor = FALSE, subtitleLabSize = 0, legendPosition = "none")
})

Volcano_Fun <- function(Volcano, no_of_rows, no_of_cols) {
    patchwork::wrap_plots(Volcano, 
    nrow = no_of_rows, ncol = no_of_cols)
}

Volcano_Patch <- Volcano_Fun(Volcano, 2, 4)

ggsave("Plots/Volcano_WRSum_filtered.svg", plot = Volcano_Patch, units = "cm", dpi = 300, width = 42, height = 25, limitsize = FALSE)

## DESeq2 ALDO vs. CTRL (FindMarkers)

In [ ]:
cluster_annotations_LA <- list(
    "0" = "CM_0",
    "1" = "EC-end",
    "2" = "FB",
    "3" = "MΦ",
    "4" = "EC-cap",
    "5" = "PER",
    "6" = "MESO",
    "9" = "CM_1",
    "10" = "TC",
    "11" = "EC-lym",
    "12" = "SC",
    "13" = "CM_2",
    "14" = "BC"
)

In [ ]:
doAnalysis <- function(LA_LV, cluster_annotations){
    seurat_table = subset(x = obj_subset, subset = chamber == LA_LV)
    seurat_table <- NormalizeData(seurat_table)
    seurat_table <- ScaleData(seurat_table)
    #Assign "treatment" as ident
    Idents(seurat_table) <- seurat_table@meta.data$"treatment"
        
    table_list <- lapply(names(cluster_annotations), function(cl){
        name_of_clust = cluster_annotations[[cl]]
        my_subseur <- subset(x = seurat_table, subset = seurat_clusters == cl)
        my_markers <- FindMarkers(my_subseur, ident.1 = "ALDO", ident.2 = "CTRL", assay = "RNA", test.use = "DESeq2", pseudocount.use = 1, logfc.threshold = 0.1)
        write.csv(my_markers, file = paste0("DEGs/ALDOvsCTRL DESeq2/degs_DESeq2", "_", LA_LV, "_", name_of_clust, ".csv"))
        return(my_markers)
    })
    names(table_list) <- cluster_annotations
    
    return(table_list)
}

In [ ]:
doAnalysis("LA", cluster_annotations_LA)

In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.1 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/scrna_dm/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] pals_1.9                    viridis_0.6.5              
 [3] viridisLite_0.4.2           svglite_2.1.3              
 [5] Rsamtools_2.18.0            Biostrings_2.70.3          
 [7] XVector_0.42.0              EnhancedVolcano_1.20.0     
 [9] ggrepel_0.9.6   